### drive check

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

BASE_URL = "https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfPMN/pmn.cfm"

options = webdriver.ChromeOptions()
#options.add_argument("--headless")  # 확인용이면 주석처리해서 브라우저 띄워도 됨
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                     "AppleWebKit/537.36 (KHTML, like Gecko) "
                     "Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get(BASE_URL)
time.sleep(3)  # 페이지 로딩 기다림

# 페이지 소스 출력
html = driver.page_source
print(html[:2000])   # 앞부분만 출력 (길면 2000자 정도 확인)

driver.quit()

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:addthis="http://www.addthis.com/help/api-spec"><head><link rel="stylesheet" type="text/css" href="//s7.addthis.com/static/r07/widget71.css" media="all">

  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  

  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
  <meta http-equiv="X-UA-Compatible" content="IE=7">


  <script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-B0V4T2XCW7&amp;cx=c&amp;gtm=4e59f1"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-B0V4T2XCW7"></script><script type="text/javascript" src="/scripts/includes/js/wcm.toggle.js"></script>
  <script type="text/javascript" src="/scripts/includes/js/ssajax_2012.js"></script>



  

  <title>510(k) Premarket Notification</title>

  <!-- /**** Begin CSS References ****/-->

  
    <link href="/scripts/cdrh/cfdocs/default_style.css" rel="stylesheet" media="all" type="text/c

### main

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time, random, json, requests, fitz
from urllib.parse import urljoin

BASE_URL = "https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfPMN/pmn.cfm"

def setup_driver():
    options = webdriver.ChromeOptions()
    # 필요하면 headless 켜기
    # options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) "
                         "Chrome/120.0.0.0 Safari/537.36")
    # PDF를 브라우저에서 바로 열어도 상관없지만, 다운로드 강제 X (cookies로 requests 접근할거라 무관)
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def requests_session_from_driver(driver):
    s = requests.Session()
    # UA/Referer 세팅 (서버 쪽 탐지 완화)
    s.headers.update({
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/120.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,application/pdf;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.8",
        "Connection": "keep-alive",
    })
    for c in driver.get_cookies():
        try:
            s.cookies.set(c['name'], c['value'], domain=c.get('domain'), path=c.get('path', '/'))
        except Exception:
            # domain이 None이면 그냥 기본으로 세팅
            s.cookies.set(c['name'], c['value'])
    return s

def extract_pdf_text_with_session(session, pdf_url, referer=None):
    try:
        if referer:
            session.headers.update({"Referer": referer})
        # redirect 따라가되, apology 페이지 탐지
        r = session.get(pdf_url, allow_redirects=True, timeout=60)
        final_url = r.url
        if "apology_objects" in final_url.lower():
            print(f"[apology 차단] {final_url}")
            return None
        # content-type 체크
        ctype = r.headers.get("Content-Type","").lower()
        if not (final_url.lower().endswith(".pdf") or "application/pdf" in ctype):
            print(f"[PDF 아님] {final_url} (Content-Type={ctype})")
            return None

        with fitz.open(stream=r.content, filetype="pdf") as doc:
            text = "\n".join([p.get_text("text") for p in doc])
        text = text.strip()
        if not text:
            print(f"[텍스트 없음] {final_url}")
            return None
        time.sleep(random.uniform(1.2, 2.0))
        return text
    except Exception as e:
        print(f"[PDF 추출 실패] {pdf_url}: {e}")
        return None

def collect_page_rows(driver):
    """현재 결과 페이지에서 (K-number, device, applicant, date, detail_link) 리스트로 먼저 수집"""
    items = []
    # 결과 테이블 대기 (클래스가 바뀔 수 있어 table 전체로 대기)
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//table"))
    )
    # 결과 행들 선택: 첫 번째 헤더행 제외
    rows = driver.find_elements(By.XPATH, "//table//tr[td/a[contains(@href,'pmn.cfm?ID=')]]")
    for row in rows:
        try:
            # 각 칼럼 안전 추출
            k_a = row.find_element(By.XPATH, ".//td[1]/a")
            k_number = k_a.text.strip()
            detail_link = k_a.get_attribute("href")
            device = row.find_element(By.XPATH, ".//td[2]").text.strip()
            applicant = row.find_element(By.XPATH, ".//td[3]").text.strip()
            decision_date = row.find_element(By.XPATH, ".//td[4]").text.strip()
            items.append({
                "k_number": applicant,
                "device_name": k_number,
                "applicant": applicant,
                "decision_date": decision_date,
                "detail_link": detail_link
            })
        except Exception:
            continue
    return items

def crawl_device(driver, device_name, max_pages=1):
    results = []
    driver.get(BASE_URL)
    wait = WebDriverWait(driver, 15)

    # 검색어 입력
    box = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='DeviceName']")))
    box.clear()
    box.send_keys(device_name)
    driver.find_element(By.XPATH, "//input[@value='Search']").click()

    time.sleep(random.uniform(2.0, 3.0))

    page = 1
    while True:
        print(f"📄 {device_name} / page={page}")
        # 현재 페이지의 결과 링크들 먼저 수집
        page_items = collect_page_rows(driver)
        print(f"  - rows: {len(page_items)}")

        # 쿠키 세션 준비 (요청마다 재사용)
        sess = requests_session_from_driver(driver)

        # 각 상세 페이지 방문 → Summary → PDF 텍스트 추출
        for it in page_items:
            # 상세로 이동
            driver.get(it["detail_link"])
            time.sleep(random.uniform(1.2, 2.2))

            summary_link, summary_text = None, None
            try:
                # Summary 링크 (대소문자 무시)
                # "Summary (English)" 같은 변형도 대응
                summary_el = WebDriverWait(driver, 8).until(
                    EC.presence_of_element_located((
                        By.XPATH,
                        "//a[contains(translate(normalize-space(.),"
                        "'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'summary')]"
                    ))
                )
                href = summary_el.get_attribute("href")
                summary_link = urljoin(driver.current_url, href)

                # **중요**: Selenium 세션 쿠키로 requests 접근
                summary_text = extract_pdf_text_with_session(
                    sess, summary_link, referer=driver.current_url
                )

            except Exception as e:
                print(f"[Summary 링크 없음/실패] {it['k_number']}: {e}")

            results.append({
                **it,
                "summary_link": summary_link,
                "summary_text": summary_text
            })

            # 결과 목록으로 back
            driver.back()
            time.sleep(random.uniform(1.0, 1.8))

        # 다음 페이지
        if page >= max_pages:
            break
        try:
            next_btn = driver.find_element(By.LINK_TEXT, "Next 10")
            next_btn.click()
            page += 1
            time.sleep(random.uniform(2.5, 4.0))
        except Exception:
            break

    return results

def main():
    driver = setup_driver()
    try:
        data = crawl_device(driver, "implant", max_pages=50)  # 테스트면 max_pages 1로 설정할것
    finally:
        driver.quit()

    with open("fda_implant_test.jsonl", "w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"✅ implant {len(data)}개 저장 완료")

if __name__ == "__main__":
    main()

📄 implant / page=1
  - rows: 10
[Summary 링크 없음/실패] Vaccess™ CT Low-Profile Power-Injectable Implantable Port; Vaccess™ CT Power-Injectable Implantable Port; PowerPort™ duo M.R.I. ™ Implantable Port: Message: 
Stacktrace:
	GetHandleVerifier [0x0xd4d2a3+66419]
	GetHandleVerifier [0x0xd4d2e4+66484]
	(No symbol) [0x0xb24bd3]
	(No symbol) [0x0xb6e958]
	(No symbol) [0x0xb6ecfb]
	(No symbol) [0x0xbb5152]
	(No symbol) [0x0xb91064]
	(No symbol) [0x0xbb28a1]
	(No symbol) [0x0xb90e16]
	(No symbol) [0x0xb625ce]
	(No symbol) [0x0xb634a4]
	GetHandleVerifier [0x0xf95ee3+2461619]
	GetHandleVerifier [0x0xf90f66+2441270]
	GetHandleVerifier [0x0xd76242+234258]
	GetHandleVerifier [0x0xd66208+168664]
	GetHandleVerifier [0x0xd6d1ad+197245]
	GetHandleVerifier [0x0xd555f8+100040]
	GetHandleVerifier [0x0xd55792+100450]
	GetHandleVerifier [0x0xd3f74a+10266]
	BaseThreadInitThunk [0x0x7571fcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x777982ae+286]
	RtlGetAppContainerNamedObjectPath [0x0x7779827e+238]



InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=140.0.7339.128); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0xd4d2a3+66419]
	GetHandleVerifier [0x0xd4d2e4+66484]
	(No symbol) [0x0xb24bd3]
	(No symbol) [0x0xb14070]
	(No symbol) [0x0xb3277b]
	(No symbol) [0x0xb97a45]
	(No symbol) [0x0xbb21b9]
	(No symbol) [0x0xb90e16]
	(No symbol) [0x0xb625ce]
	(No symbol) [0x0xb634a4]
	GetHandleVerifier [0x0xf95ee3+2461619]
	GetHandleVerifier [0x0xf90f66+2441270]
	GetHandleVerifier [0x0xd76242+234258]
	GetHandleVerifier [0x0xd66208+168664]
	GetHandleVerifier [0x0xd6d1ad+197245]
	GetHandleVerifier [0x0xd555f8+100040]
	GetHandleVerifier [0x0xd55792+100450]
	GetHandleVerifier [0x0xd3f74a+10266]
	BaseThreadInitThunk [0x0x7571fcc9+25]
	RtlGetA

### text extraction validation

In [47]:
import json

with open("fda_implant_test.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        k_number = item.get("k_number")
        summary_text = item.get("summary_text")

        if summary_text:  # None 아닌 경우만
            print(f"{k_number}: {len(summary_text)} chars")
        else:
            print(f"{k_number}: summary_text is None")

Vaccess™ CT Low-Profile Power-Injectable Implantable Port; Vaccess™ CT Power-Injectable Implantable Port; PowerPort™ duo M.R.I. ™ Implantable Port: summary_text is None
NobelZygoma TiUltra Implant system: 91755 chars
MIS C1 Implant System, MIS Seven Implant System, MIS M4 Implant System, MIS Lance+ Implant System, MIS Lance+ Conical Connection System: 28818 chars
Treace Medical Concepts (TMC) Implant Fixation System: 13930 chars
IPD Dental Implant Abutments: 37192 chars
Neodent Implant System - Zirconia Implant System: 55691 chars
S.I.N. Dental Implant System: 27459 chars
Straumann® RidgeFit Implants: 21567 chars
Fusion Craniofacial Implant; Fusion Skull Implant: 17926 chars
MIS LYNX Conical Connection Implant System: 30287 chars
